In [1]:
import glob 
import os,re
import yt_dlp
import json
import httplib2
import openai
import requests
import pinecone 
import pandas as pd
import numpy as np
import urllib.request
from langchain import OpenAI
from bs4 import BeautifulSoup
from bs4.element import Comment
from langchain.llms import OpenAIChat
from langchain.chains import VectorDBQA
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma
from bs4 import BeautifulSoup, SoupStrainer
from langchain.vectorstores import Pinecone
from youtubesearchpython import ChannelsSearch
from langchain.chains import VectorDBQAWithSourcesChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

/Users/31treehaus/opt/anaconda3/envs/ml/lib/python3.9/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Get video metadata - 
 
* https://pypi.org/project/youtube-search-python/
* `Problem: Only gets first 100!` 

In [2]:
# get channel 
channelsSearch = ChannelsSearch('lexfridman', limit = 10, region = 'US')
r = channelsSearch.result()
id = r['result'][0]['id']

# get all videos
channel_id = "UCSHZKyawb77ixDdsGog4iWA"
playlist = Playlist(playlist_from_channel_id(channel_id))
print(f'Videos Retrieved: {len(playlist.videos)}')

# store
stor_metadata=pd.DataFrame()
for v in playlist.videos:
    stor_metadata.loc[v['title'],'link']=v['link']

Videos Retrieved: 100


Download audio - 

* https://github.com/yt-dlp/yt-dlp/wiki/Installation
* https://github.com/yt-dlp/yt-dlp

In [ ]:
ydl_opts = {
    'format': 'm4a/bestaudio/best',
    # ℹ️ See help(yt_dlp.postprocessor) for a list of available Postprocessors and their arguments
    'postprocessors': [{  # Extract audio using ffmpeg
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'm4a',
    }]
}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    error_code = ydl.download(str(url))

`Transcribe`

* `Problem`: `Maximum content size limit (26214400) exceeded (56033653 bytes read)` w/ API
* Need to do it manually on GPU machine

In [ ]:
fis = glob.glob("audio/*m4a")
fis_s=sorted(fis)
fis_s[1]

file = open(fis_s[0], "rb")
transcription = openai.Audio.transcribe("whisper-1", file)
print(transcription)

`Read existing transcriptions`

* https://karpathy.ai/lexicap/index.html
* https://langchain.readthedocs.io/en/latest/modules/indexes/chain_examples/qa_with_sources.html

In [6]:
# Get all pages 
http = httplib2.Http()
status, response = http.request("https://karpathy.ai/lexicap/")
links = []
for link in BeautifulSoup(response, 'html.parser', parse_only=SoupStrainer('a')):
    if link.has_attr('href'):
        links.append(link['href'])
links_tx = ["https://karpathy.ai/lexicap/"+l for l in links if "0" in l]
links_tx[0:3]

['https://karpathy.ai/lexicap/0001-large.html',
 'https://karpathy.ai/lexicap/0002-large.html',
 'https://karpathy.ai/lexicap/0003-large.html']

In [7]:
# Get text -
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(string=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def get_text_and_title(url):
    html = urllib.request.urlopen(url).read()
    t=(text_from_html(html))
    title=t.split("|")[0].split("back to index")[1].strip()
    return t, title

# Get links -
def get_links(URL):
    http = httplib2.Http()
    status, response = http.request(URL)
    links = []
    for link in BeautifulSoup(response, 'html.parser', parse_only=SoupStrainer('a')):
        if link.has_attr('href'):
            links.append(link['href'])
    links_clean = [l for l in links if "https" in l]
    return links_clean

# Get image -
def get_img(URL,title,episode_id):
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    img_tags = soup.find_all('img')
    urls = [img['src'] for img in img_tags]
    for url in urls:
        response = requests.get(url)
        imgpath="img/%s.jpg"%episode_id
        with open(imgpath, 'wb') as f:
            if 'http' not in url:
                # sometimes an image source can be relative 
                # if it is provide the base url which also happens 
                # to be the site variable atm. 
                url = '{}{}'.format(site, url)
            response = requests.get(url)
            f.write(response.content)
    return imgpath

# Full pipeline - 
def pre_process(URL,episode_id):

    t,title=get_text_and_title(URL)
    links=get_links(URL)
    img=get_img(URL,title,episode_id)
    stor_chunk = pd.DataFrame()
    stor_chunk['chunks']= t.split("link |")
    stor_chunk['clean_chunks']=stor_chunk['chunks'].apply(lambda x: re.sub(r"[^a-zA-Z ]+", '', x)).apply(lambda x: x.strip())
    stor_chunk['links']=links
    all_text = stor_chunk['clean_chunks'].str.cat(sep=' ')
    return all_text, links, title

def make_splits(chunks,URL):

    # ID
    episode_id=URL.split("/")[-1].split("-")[0]

    # Pre-processing
    texts,links,title=pre_process(URL,episode_id)
    
    # Splits 
    # text_splitter = CharacterTextSplitter(chunk_size=chunks, chunk_overlap=50, separator=" ") 
    # texts = text_splitter.split_text(texts)
    # print(len(texts)) 
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunks, chunk_overlap=50) 
    texts_recusive = text_splitter.split_text(texts)
    print(len(texts_recusive)) 

    # Metadata 
    N = len(texts_recusive) 
    bins = np.linspace(0, len(links)-1, N, dtype=int)
    sampled_links = [links[i] for i in bins]
    # Here we can add "link", "title", etc that can be fetched in the app 
    metadatas=[{"source":title + " " +link,"id":episode_id,"link":link,"title":title} for link in sampled_links]
    print(len(metadatas))

    return texts_recusive,metadatas,title,episode_id

# *** Chunk size: key parameter *** 
chunks = 1500
# *** Chunk size: key parameter *** 
splits_all = [ ]
metadatas_all = [ ]
titles = [ ]
 
# Iterate 
stor=pd.DataFrame()
for page in links_tx:
    # try:
    print("Writing: %s"%page)
    splits,metadatas,title,episode_id=make_splits(chunks,page)
    stor.loc[episode_id,'title']=title 
    with open('docs/%s.txt'%episode_id, "w") as f:
        for string in splits:
            f.write(string + "\n") 
    f.close()
    with open('metadatas/%s.json'%episode_id, "w") as f:
        json.dump(metadatas, f)
    f.close()
    splits_all.append(splits)
    metadatas_all.append(metadatas)
    # except:
    #    print("Error on page: %s"%page)
    
# Combine all 
splits_combined = []
for sublist in splits_all:
    splits_combined.extend(sublist)
metadatas_combined = []
for sublist in metadatas_all:
    metadatas_combined.extend(sublist)

Writing: https://karpathy.ai/lexicap/0001-large.html
41
41
Writing: https://karpathy.ai/lexicap/0002-large.html
32
32
Writing: https://karpathy.ai/lexicap/0003-large.html
19
19
Writing: https://karpathy.ai/lexicap/0004-large.html
19
19
Writing: https://karpathy.ai/lexicap/0005-large.html
20
20
Writing: https://karpathy.ai/lexicap/0006-large.html
34
34
Writing: https://karpathy.ai/lexicap/0007-large.html
52
52
Writing: https://karpathy.ai/lexicap/0008-large.html
17
17
Writing: https://karpathy.ai/lexicap/0009-large.html
38
38
Writing: https://karpathy.ai/lexicap/0010-large.html
23
23
Writing: https://karpathy.ai/lexicap/0011-large.html
34
34
Writing: https://karpathy.ai/lexicap/0012-large.html
33
33
Writing: https://karpathy.ai/lexicap/0013-large.html
31
31
Writing: https://karpathy.ai/lexicap/0014-large.html
32
32
Writing: https://karpathy.ai/lexicap/0015-large.html
31
31
Writing: https://karpathy.ai/lexicap/0016-large.html
40
40
Writing: https://karpathy.ai/lexicap/0017-large.html
48


`Read in the saved files`

In [2]:
def get_saved_doc(doc):

    dir_path, file_name = os.path.split(doc)
    name, ext = os.path.splitext(file_name)

    with open(doc, "r") as f:
        my_strings = f.readlines()
    f.close()

    doc_to_read_txt = [string.strip() for string in my_strings]

    with open('metadatas/%s.json'%name, "r") as f:
        doc_to_read_metadatas = json.load(f)
    f.close()

    return doc_to_read_txt,doc_to_read_metadatas

# Read files in 
docs = glob.glob("docs/*")
splits_all_read_fromdisk = []
metadatas_all_readfromdisk = []
for doc_to_read in docs:
    doc_to_read_txt,doc_to_read_metadatas = get_saved_doc(doc_to_read)
    splits_all_read_fromdisk.append(doc_to_read_txt)
    metadatas_all_readfromdisk.append(doc_to_read_metadatas)

# Combine 
splits_combined_fromdisk = []
metadatas_combined_fromdisk = []
for sublist in splits_all_read_fromdisk:
    splits_combined_fromdisk.extend(sublist)
for sublist in metadatas_all_readfromdisk:
    metadatas_combined_fromdisk.extend(sublist)

In [4]:
print(len(splits_combined_fromdisk))
print(splits_combined_fromdisk[0])
print(len(metadatas_combined_fromdisk))
print(metadatas_combined_fromdisk[0])


20632
back to index Nic Carter Bitcoin Core Values Layered Scaling and Blocksize Debates  Lex Fridman Podcast  small model  large model The following is a conversation with Nick Carter who is a partner at Castle Island Ventures cofounder of CoinMetricsio and previously a crypto asset research analyst at Fidelity Investments Hes a prominent writer speaker and podcaster on topics around decentralized finance and especially Bitcoin Quick mention of our sponsors The Information Athletic Greens Four Sigmatic and Blinkist Check them out in the description to support this podcast This conversation with Nick Carter is part of a series of episodes on cryptocurrency that is a small journey of exploration Im on because I find decentralized finance and especially Bitcoin fascinating technically and philosophically especially because it may be the very mechanism that achieves a global decentralization of power giving more sovereignty to the individual and making our systems more resilient to corrup

In [8]:
print(len(splits_combined))
print(splits_combined[0])
print(len(metadatas_combined))
print(metadatas_combined[0])

20632
back to index Max Tegmark Life   Lex Fridman Podcast  small model  large model As part of MIT course S Artificial General Intelligence Ive gotten the chance to sit down with Max Tegmark He is a professor here at MIT Hes a physicist spent a large part of his career studying the mysteries of our cosmological universe But hes also studied and delved into the beneficial possibilities and the existential risks of artificial intelligence Amongst many other things he is the cofounder of the Future of Life Institute author of two books both of which I highly recommend First Our Mathematical Universe Second is Life Hes truly an out of the box thinker and a fun personality so I really enjoy talking to him If youd like to see more of these videos in the future please subscribe and also click the little bell icon to make sure you dont miss any videos Also Twitter LinkedIn agimitedu if you wanna watch other lectures or conversations like this one Better yet go read Maxs book Life Chapter seve

In [29]:
oriol_text,oriol_metadatas=get_saved_doc('docs/0306.txt')
demis_text,demis_metadatas=get_saved_doc('docs/0299.txt')
txt_test=oriol_text+demis_text
metadatas_test=oriol_metadatas+demis_metadatas
print(len(oriol_text))
print(len(oriol_metadatas))
print(len(txt_test))
print(len(metadatas_test))

62
62
133
133


`Embedding: small-scale test`

In Python, we have many options:

https://langchain.readthedocs.io/en/latest/reference/modules/vectorstore.html

Only Pinecone, Chroma can be used w/ JS: 

https://hwchase17.github.io/langchainjs/docs/modules/indexes/vector_stores/supabase/

Let's test a small DB w/ Pinecone and Chroma.

In [30]:
# Test for Chroma 
embeddings = OpenAIEmbeddings()
c_test = Chroma.from_texts(txt_test, embeddings, metadatas=metadatas_test)

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


In [31]:
embeddings = OpenAIEmbeddings()
f_test = FAISS.from_texts(txt_test, embeddings, metadatas=metadatas_test)

In [24]:
# Test for pinecone 
pinecone.init(
    api_key="66b41af0-4796-4bae-84a0-2409e6babab6",  
    environment="us-east1-gcp"  
)
index_name = "lex-test"
embeddings = OpenAIEmbeddings()
# p_test = Pinecone.from_texts(txt_test, embeddings, index_name=index_name, metadatas=metadatas_test)
p_test = Pinecone.from_existing_index(index_name=index_name,embedding=embeddings)

`Embed full dataset`

* Took 1hr to index ~6k vectors (44% of the data)
* See `JSONDecodeError` with Chroma and FAISS; needs JSON strings 
* But it is not reproduced; e.g., I did not see it the second time I tried FAISS

https://docs.pinecone.io/docs/insert-data
https://langchain.readthedocs.io/en/latest/modules/indexes/how_to_guides.html
https://langchain.readthedocs.io/en/latest/reference/modules/vectorstore.html

In [10]:
for i, dict in enumerate(metadatas_combined):
    
    try:
        # encode the dictionary into a JSON string
        json_str = json.dumps(dict)
        # decode the JSON string into a dictionary
        # json_obj = json.loads(json_str)
    except json.JSONDecodeError as e:
        print(f"JSONDecodeError in object {i}: {e}")

metadatas_combined_json_str=[json.dumps(dict) for dict in metadatas_combined]

In [170]:
faiss_docstore = FAISS.from_texts(splits_combined, embeddings, metadatas=metadatas_combined)
faiss_docstore.save_local('index/lex_faiss')

Exiting: Cleaning up .chroma directory


~ 1 hr to hit `index 55`

In [12]:
# Write final settings to Pinecone for prod 
pinecone.init(
    api_key="66b41af0-4796-4bae-84a0-2409e6babab6",  
    environment="us-east1-gcp"  
)
index_name = "lex-gpt"
embeddings = OpenAIEmbeddings()
# p = Pinecone.from_texts(splits_combined[0:2], embeddings, index_name=index_name, metadatas=metadatas_combined[0:2])
p = Pinecone.from_existing_index(index_name=index_name,embedding=embeddings)

In [ ]:
# Add all data 
import math, time 
chunk_size = 100
num_chunks = math.ceil(len(splits_combined) / chunk_size)
for i in range(last_chunk,num_chunks):
    
    print(i)
    start_time = time.time()
    # try:

    # Calculate the start and end indices for the current chunk
    start_idx = i * chunk_size
    end_idx = min(start_idx + chunk_size, len(splits_combined))

    # Extract the current chunk
    current_splits = splits_combined[start_idx:end_idx]
    current_metadatas = metadatas_combined[start_idx:end_idx]

    # Add the current chunk to the vector database
    p.add_texts(texts = current_splits, metadatas=current_metadatas)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")
    print("--------")

    # except:

    #    print("Error at chunk: %s"%i)
    #    time.sleep(5)

# docsearch_demis = Pinecone.from_texts(splits_combined, embeddings, index_name=index_name, metadatas=metadatas_combined)

In [ ]:
print("CHROMA")
embeddings = OpenAIEmbeddings()
chroma_docstore = Chroma.from_texts(splits_combined, embeddings, metadatas=metadatas_combined_json_str,persist_directory="index/",collection_name="lex-gpt")


In [89]:
print(len(splits_combined))
print(len(metadatas_combined))
lex_index = Pinecone.from_existing_index(index_name=index_name,embedding=embeddings)

15296
15296


`Run chain`

https://langchain.readthedocs.io/en/latest/modules/indexes/chain_examples/qa_with_sources.html#
https://langchain.readthedocs.io/en/latest/modules/indexes/chain_examples/vector_db_qa_with_sources.html

In [26]:
import time
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

def run_qa_chain(llm,query,docstore):

    start_time = time.time()
    docs = docstore.similarity_search(query)
    chain = load_qa_chain(llm, chain_type="stuff")
    a = chain.run(input_documents=docs, question=query)
    print(a["output_text"])
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")
    print("--------")

def run_qa_sources_chain(llm,query,docstore):
    
    start_time = time.time()
    docs = docstore.similarity_search(query)
    chain = load_qa_with_sources_chain(llm, chain_type="stuff")
    a = chain({"input_documents": docs, "question": query})
    print(a["output_text"])
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")
    print("--------")

def run_vectordbqa_chain(llm,query,docstore):

    start_time = time.time()
    chain = VectorDBQA.from_chain_type(llm, chain_type="stuff", vectorstore=docstore)
    a = chain({"query": query})
    print(a['result'])
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")
    print("--------")

def run_vectordb_sources_chain(llm,query,docstore):

    start_time = time.time()
    chain = VectorDBQAWithSourcesChain.from_chain_type(llm, chain_type="stuff", vectorstore=docstore)
    a = chain({"question": query},return_only_outputs=True)
    print(a["answer"])
    print(a["sources"])
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")
    print("--------")

llm = OpenAIChat(temperature=0)

# run_qa_chain(llm,q,p_oriol)
# run_qa_sources_chain(llm,q,p_oriol) 
# run_vectordbqa_chain(llm,q,p_oriol)
#run_vectordb_sources_chain(llm,q,p_test)
#run_vectordb_sources_chain(llm,q,p)


In [27]:
q = "What does the Gato model do?"
run_vectordb_sources_chain(llm,q,p_test)
run_vectordb_sources_chain(llm,q,p)

The Gato model is a transformer neural network that models sequences of actions, words, and observations, and is trained through a massive imitation learning algorithm that imitates data sets of observations. It is a general agent that can take actions in an environment and generate the next step. It can be thought of as inputs of images, text, video, and actions, and outputs words and actions. One possible next step for the Gato model is to convert even images into language through a crude semantic segmentation. 

Oriol Vinyals: Deep Learning and Artificial General Intelligence https://www.youtube.com/watch?v=aGBLRlLe7X8&t=1461, https://www.youtube.com/watch?v=aGBLRlLe7X8&t=1557, https://www.youtube.com/watch?v=aGBLRlLe7X8&t=2715
Elapsed time: 13.23695421218872 seconds
--------
The Gato model is a transformer neural network that models sequences of actions, words, and observations, and is trained through a massive imitation learning algorithm that imitates data sets of observations. I

In [32]:
q = "What is the future path for AGI?"
#run_vectordb_sources_chain(llm,q,p_test)
#run_vectordb_sources_chain(llm,q,p)
run_vectordb_sources_chain(llm,q,c_test)
run_vectordb_sources_chain(llm,q,f_test)

The future path for AGI involves creating benchmarks to test generalizability and scaling up models to be more powerful. Language is likely to be a key area where AGI will shine. There are still breakthroughs to be made, but progress is being made towards creating a single model that can be taught new tasks interactively. The Turing test is still seen as a critical benchmark for AGI. 

Oriol Vinyals: Deep Learning and Artificial General Intelligence https://www.youtube.com/watch?v=aGBLRlLe7X8&t=7665, Oriol Vinyals: Deep Learning and Artificial General Intelligence https://www.youtube.com/watch?v=aGBLRlLe7X8&t=3696, Oriol Vinyals: Deep Learning and Artificial General Intelligence https://www.youtube.com/watch?v=aGBLRlLe7X8&t=1230, Demis Hassabis: DeepMind - AI, Superintelligence & the Future of Humanity https://www.youtube.com/watch?v=Gfr50f6ZBvo&t=368
Elapsed time: 15.502749919891357 seconds
--------
The future path for AGI involves interactive teaching and the development of more powe

In [33]:
run_qa_sources_chain(llm,q,c_test)
run_qa_sources_chain(llm,q,f_test)

The future path for AGI involves creating benchmarks to test generalizability and scaling up models to be more powerful. Language is likely to be a key area where AGI will shine. There are still breakthroughs to be made, but progress is being made towards a single model that can be taught new tasks interactively. The Turing test is still seen as a critical benchmark for AGI. 
SOURCES: Oriol Vinyals: Deep Learning and Artificial General Intelligence https://www.youtube.com/watch?v=aGBLRlLe7X8&t=7665, Oriol Vinyals: Deep Learning and Artificial General Intelligence https://www.youtube.com/watch?v=aGBLRlLe7X8&t=3696, Oriol Vinyals: Deep Learning and Artificial General Intelligence https://www.youtube.com/watch?v=aGBLRlLe7X8&t=1230, Demis Hassabis: DeepMind - AI, Superintelligence & the Future of Humanity https://www.youtube.com/watch?v=Gfr50f6ZBvo&t=368
Elapsed time: 16.62952184677124 seconds
--------
The future path for AGI involves interactive teaching and the development of more powerf